# GP Cartpole Server

This notebook introduces the genetic programming (GP) approach for the cartpole REST interface. GP creates a program for the interface's default controllers. It enables the cartpole REST interface to reply with a concrete response to a client's requests.

The figure illustrates the position of the GP software within the REST interface implementation.  

![REST gp design](http://www.plantuml.com/plantuml/png/5SWx3i8m303Ggy05uW4cPjYO2dlLEbOQ9R4ZVyJrKT_uhfMnHwRGVZdSfpq0TFkbKls8FOmKXPtO4ye0p8Jjfl1StlVbzVd2sCn9WSLf5sa6vwvbhfci5aun-Xy0)

The [OpenAPI generator](https://github.com/OpenAPITools/openapi-generator) creates a server stub consisting of the REST interface and the REST controller prototype using the [cartpole API spec](https://raw.githubusercontent.com/cdeck3r/DRL4REST/master/src/cartpole/spec/OpenAPIv1.json). In this approach we replace the controller's empty function body with an implementation found through genetic programming. Additionally, we introduce a ServerModel, which keeps a state if required. One may access the state using CRUD operations. 

In contrast to mocking solutions, e.g. [Prism](https://github.com/stoplightio/prism), GP enables more complex behavior than just a compliant response to a client's request.

## Example: `cart_get()` Controller Function

As a example for the REST controller, look at the `cart_get()` controller prototype the OpenAPI generator created for the cartpole REST interface. The generator nicely separates the controller from all the marshalling and unmarshalling operations for data transport from/to the client.

```python
def cart_get():  # noqa: E501
    """cart_get

     # noqa: E501


    :rtype: Cart
    """
    return 'do some magic!'
```

A concrete example implementation would look like the following:

```python
def cart_get():  # noqa: E501
    # just return a Cart object with random values

    position = random()
    velocity = random()
    direction = choice(["left", "right"]) 

    return Cart(position=position, velocity=velocity, direction=direction)
```

The `Cart` class is one the generated models. The constructor defines the params as follows:
```python
def __init__(self, position=None, velocity=None, direction=None):  # noqa: E501
    """Cart - a model defined in OpenAPI

    :param position: The position of this Cart.  # noqa: E501
    :type position: float
    :param velocity: The velocity of this Cart.  # noqa: E501
    :type velocity: float
    :param direction: The direction of this Cart.  # noqa: E501
    :type direction: str
    """
    ...
```

## GP Problem Formulation

For the `cart_get()` example from above, we envision the following controller implementation: if there is already a cart object, then the controller will return this object. Otherwise, the controller creates a new one and returns it. Here's the pseudo code:

```python
if cart exists 
    return cart
else
    return create_cart()
```
**The GP problem is: Synthesize a program with a behavior like the one above.**

We test run the synthesized code within the `evaluate()` function of the GP algorithm. It is successfull, if

1. `cart_get()` controller runs without any exception 
1. `cart_get()` controller returns a non-null result 
1. `cart_get()` controller returns a non-empty result 
1. repeated `cart_get()` requests result in the same response

The `evaluate()` function scores each GP created programs for the program evolution.

## Import libs

DEAP was previously installed in this image. If not, run `pip install deap`.

In [1]:
import random

import numpy

from functools import partial

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

The OpenAPI generated interface code is based on python-flask. If necessary, activate the following cell `ESC + y` and install the requirements.

In [2]:
%%bash
# define project environment
PROJECT_DIR="/DRL4REST"
OPENAPI_SERVER_DIR="$PROJECT_DIR/openapi/cartpole/python-flask"

# install requirements
cd "$OPENAPI_SERVER_DIR" || exit
pip install -r requirements.txt || exit
pip install -r test-requirements.txt || exit

# install other requirements
pip install Werkzeug==0.16.1 || exit

Ignoring connexion: markers 'python_version == "3.5"' don't match your environment
Ignoring connexion: markers 'python_version == "3.4"' don't match your environment
Ignoring connexion: markers 'python_version <= "2.7"' don't match your environment
  Created wheel for flask-testing: filename=Flask_Testing-0.6.1-py3-none-any.whl size=7639 sha256=04242749c4969c3ca32d54273667b09c8a3e14f206a3d0b042393bb4cc8cf429
  Stored in directory: /home/jovyan/.cache/pip/wheels/d3/a7/7f/effdb220813fd331ca8d1961b4fd629fa6dcd54c9cdc83eaeb
Successfully built flask-testing
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 1.0.1
    Uninstalling Werkzeug-1.0.1:
      Successfully uninstalled Werkzeug-1.0.1


In [3]:
# set path to import code generated by OpenAPI
import sys
sys.path.append("/DRL4REST/openapi/cartpole/python-flask")

## GP Software Workflow

The OpenAPI generator creates default controllers and test. We replace the default controller by the GP one, called `GP_Controller`, and the extend the default tests by `GP_TestDefaultController`. The following activity diagram displays how the REST part in embedded in the actions of the genetic programming framework.

![GP software activity diagram](http://www.plantuml.com/plantuml/png/1S4zhi8m303WgxntWNY0YHd4me0AUvKwLciriP5_GDoUx_jdmPwDqlhV5l7o4u3LJravjbpIoL0bIAAW3i16k1yCpjFjE5qUJnZLWmn4WnRLtU5wdnivhp9N37vpV4kE_WC0)

## ServerModel CRUD Functions

`CartpoleServer` is the implementation of the ServerModel. The ServerModel is a static class. Controller utilize the model to store state between successive calls. State access and manipulation via CRUD.

In [4]:
from openapi_server.models import Cart

class CartpoleServer(object):

    # state as class variable
    _cart = None

    # set the class in a defined (initial) state
    @classmethod
    def reset(cls):
        cls._cart = None
    
    @classmethod
    def create_cart(cls):
        if isinstance(cls._cart, type(None)):
            position = random.random()
            velocity = random.random()
            # we know the values from the models, e.g.
            #   models/cart.py or models/direction.py
            direction = random.choice(["left", "right"])
            cls._cart = Cart(position=position, velocity=velocity, direction=direction)
        
        return cls._cart

    @classmethod
    def read_cart(cls):
        return cls._cart
    
    @classmethod
    def update_cart(cls):
        pass

    @classmethod
    def delete_cart(cls):
        cls._cart = None

## Monkey Patching

Monkey patching changes the default behavior of a piece of code at runtime without changing its original source. For example, we want to change the behavior of the default controller function prototype `cart_get()`. Monkey patching replaces the prototype by the GP created program. When running the testcases, the GP created program gets executed instead of the default controller.

Concretely, the view function of an REST endpoint is patched to run a new controller function other than the default controller. While the openapi generator keeps controller nicely separated from the Connexion / Flask framework, it wraps them when loading the openapi spec. Basically, it applies various Python [decorators within the Operation object](https://github.com/zalando/connexion/blob/bed4b95205205861bd6014282d0f61d50231d3ac/connexion/operations/abstract.py#L340) around the controller function to add additional behavior. Note, [do not confuse it with the decorator pattern](https://wiki.python.org/moin/DecoratorPattern). As a consequence, the monkey patching approach has to unwrap the enpoint's view function at first to re-gain access to the (default) controller function. Afterwards, it replaces the code. 

In [5]:
class MonkeyPatching:
    def __init__(self, app=None):
        self._rules = []
        self._app = app
    
    @property
    def app(self):
        return self._app
    
    @app.setter
    def app(self, app):
        self._app = app
    
    # test if endpoint config exists
    def _endpoint_exists(self, url_path, method):
        self._rules = []
        
        for r in self._app.url_map.iter_rules():
            if (url_path == str(r)) and (method.upper() in r.methods):
                self._rules.append(r)
                return True
        return False

    # gets the endpoint's view function
    # returns the default controller functio
    def _unwrap(self, view_func):
        wrapped_func = view_func
        try:
            while wrapped_func.__wrapped__ :
                wrapped_func = wrapped_func.__wrapped__
        except Exception:
            # this is the default_controller_func
             return wrapped_func
    
    # replace endpoint's controller function with the one 
    # from the parameters
    def patch(self, url_path, method, controller_func):
        if self._endpoint_exists(url_path, method):
            rule = self._rules[0]
            view_func = self._app.view_functions[rule.endpoint]
            default_controller_func = self._unwrap(view_func)
            default_controller_func.__code__ = controller_func.__code__
            return True
        
        return False

## Testing the GP Controller

The `GP_TestDefaultController` overrides the tests generated by the OpenAPI generator in order to enable additional checks of the response. Furthermore, this class enables the Monkey Patching of the controller function. As a consequence, the `test_cart_get()` function from this class tests the GP generated controller function 

In [6]:
from openapi_server.test.test_default_controller import TestDefaultController
from pprint import pprint

import unittest

class GP_TestDefaultController(TestDefaultController):

    def __init__(self, mp=MonkeyPatching(), app_config_testing=True):
        super().__init__()
        self.app = super().create_app()
        self.app.config['TESTING'] = app_config_testing
        self.client = self.app.test_client()
       
        # associate with MonkeyPatching
        self.mp = mp
        self.mp.app = self.app    

    """
    Changes the behavior of a url_path with a new controller function
    
    Returns True if endpoint was found and successfully patched

    Limits: 
      * works only if method is unique for url_path 
      * does not support aliases
    """
    def endpoint_config(self, url_path, method, controller_func):
        return self.mp.patch(url_path, method, controller_func)

    # overrides the default test_cart_get()
    def test_cart_get(self):
        response = super().test_cart_get() #assert200(response)        
        self.assertEqual(response.content_type, 
                    'application/vnd.cartpole.cart+json', 
                    'Please check the openapi.yaml for unique content-type in response section'
                   )
        #print(response.headers)
        #print(response.json)
        return response.status_code


## Genetic Program Implementation

The GP bases on the Artificial Ant Problem from the [DEAP example site](https://deap.readthedocs.io/en/master/examples/gp_ant.html).

### Generic Functions

These are generic functions, e.g. for control flow and concatenating functions with each other, and the specific function for the server code. 

In [7]:
def progn(*args):
    for arg in args:
        arg()

def prog2(out1, out2): 
    return partial(progn,out1,out2)

def prog3(out1, out2, out3):     
    return partial(progn,out1,out2,out3)

def _if_then_else(condition, out1, out2):
    out1() if condition() else out2()

def if_then_else(condition, out1, out2):
    return partial(_if_then_else, condition, out1, out2)

def _if_then(condition, out1):
    if condition(): out1()
        
def if_then(condition, out1):
    return partial(_if_then, condition, out1)

def return_data(data):
    if callable(data):
        return data()
    else:
        return data

### Controller Functions

These are the functions the GP algorithm strings together utilizing the generic functions from above. The created program forms an individual and is tested by the `evaluate()` function. The controller functions utilize the CartpoleServer's CRUD functions.

In [8]:
cps = CartpoleServer
cps.reset()

In [9]:
pset = gp.PrimitiveSet("MAIN", 0)
pset.addPrimitive(if_then_else, 3)
pset.addPrimitive(if_then, 2)

pset.addPrimitive(prog2, 2)
pset.addPrimitive(prog3, 3)
pset.addPrimitive(return_data, 1)

pset.addTerminal(cps.create_cart)
pset.addTerminal(cps.read_cart)
pset.addTerminal(cps.update_cart)
pset.addTerminal(cps.delete_cart)

In [10]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Attribute generator
toolbox.register("expr_init", gp.genFull, pset=pset, min_=1, max_=2)

# Structure initializers
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr_init)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


### `evaluate()` Function

For each GP created controller function we run the `evaluate()` function. It patches the endpoint with the new controller function and runs the controller unittest. The UML diagram below illustrate the relations between the participating components.

Note: The `GP_controller` class wraps the GP created `_gp_controller` function. It is necessary because the `_gp_controller` is partial function.

![GP evaluate class diagram](http://www.plantuml.com/plantuml/png/1S4x3i9030JGgy05rWLKr20Q2XJeqSOPaMW_HjunN9_tzblN6Wl2wsubF-CaulcNtXlNc6F0rJhHcTIQw08j1vVpT3jEbyTJMXw4IpTYDYj3hlSNlhb69bAq-WS0)

The following sequence diagram depicts the interaction between the objects.

![GP evaluate sequence diagram](http://www.plantuml.com/plantuml/png/1S4z3i8m30RGgy05uW4cPX0B0ohiLUf-ebKJ6_m3r-Uz_IhSSaPQsoqHRpyHMVsLrnfBJkamLWb85DPEF8EtWz5vk1s7o-D9lNh0I3GmgMvEr_k8RsrP0wFZKxAtFm00)

In [11]:
# global gpc

class GP_Controller:
    
    _gp_controller = None
    
    @classmethod
    def set_controller_func(cls, func):
        cls._gp_controller = func

    @staticmethod
    def gp_controller_func():
        #print(getattr(sys.modules['__main__'], 'GP_Controller')._gp_controller)
        return getattr(sys.modules['__main__'], 'GP_Controller')._gp_controller()


gpc = GP_Controller

def testcase_all_correct():
    return Cart(position=1.1, velocity=2.2, direction='left')

def evalRESTController(individual):
   
    # Transform the tree expression to functionnal Python code
    gp_controller = gp.compile(individual, pset)
    # store the controller
    #global gpc
    gpc.set_controller_func(gp_controller)

    # Run the generated routine
    url_path = '/api/v1/cart'
    gp_test = GP_TestDefaultController()
    ret = gp_test.endpoint_config(url_path, 'get', gpc.gp_controller_func)
    score = 0
    try:
        gp_cart = gp_test.test_cart_get()
        score = gp_cart
    except AssertionError as error:
        score = -100
    
    return score,

In [12]:
toolbox.register("evaluate", evalRESTController)
toolbox.register("select", tools.selTournament, tournsize=7)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

In [17]:
expr = gp.genFull(pset, min_=1, max_=3)
tree = gp.PrimitiveTree(expr)
str(tree)

'prog3(prog2(delete_cart, delete_cart), return_data(create_cart), if_then_else(create_cart, create_cart, read_cart))'

In [181]:
expr

In [149]:
function = gp.compile(tree, pset)
gpc.set_controller_func(function)

In [166]:
def search_prog(n=3):
    
    success = False
    
    for i in range(n):
        expr = gp.genFull(pset, min_=1, max_=3)
        tree = gp.PrimitiveTree(expr)
        function = gp.compile(tree, pset)
        try:
            ret = function()
            if isinstance(ret, type(None)):
                continue
        except Exception:
            continue
        
        if not isinstance(ret, type(None)):
            success = True
            break

    if success:
        print(str(tree))
        return function
    

In [173]:
f = search_prog(10)

TypeError: 'NoneType' object is not callable

In [164]:
f

In [81]:
def test_func():
    if CartpoleServer.create_cart():
        return CartpoleServer.read_cart()
    else:
        CartpoleServer.update_cart()

In [82]:
type(test_func())

openapi_server.models.cart.Cart

In [94]:
p = CartpoleServer.read_cart
p = 3

def return_f(p):
    if callable(p):
        return p()
    else:
        return p

In [95]:
type(return_f(p))

int

In [108]:
url_path = '/api/v1/cart'
gp_test = GP_TestDefaultController()
ret = gp_test.endpoint_config(url_path, 'get', gpc.run_controller_func)
assert ret == True
gp_test.test_cart_get()

TypeError: 'Cart' object is not callable

### Run the GP Algorithm

In [42]:
random.seed(69)

pop = toolbox.population(n=300)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

algorithms.eaSimple(pop, toolbox, 0.5, 0.2, 40, stats, halloffame=hof)

KeyError: 'GP_Controller'

## Unit Testing the CartpoleServer

This unittest implements test methods for the CartpoleServer's CRUD functions.

In [9]:
import unittest
class Test_CartpoleServer(unittest.TestCase):
    # small test routine for the CartpoleServer
      
    def setUp(self):
        self.s=CartpoleServer
        
    def test_CartpoleServer(self):
        self.s.reset()

        c = self.s.read_cart()
        self.assertEqual(type(c), type(None))

        self.s.create_cart()
        c = self.s.read_cart()
        self.assertTrue( isinstance(type(c), type(Cart)) )

        c2 = self.s.read_cart()
        assert c == c2
        self.assertEqual(c.to_dict(), c2.to_dict())

        self.s.delete_cart()
        c = self.s.read_cart()
        self.assertEqual(type(c), type(None))

        self.s.create_cart()
        c = self.s.read_cart()
        self.assertTrue( isinstance(type(c), type(Cart)) )
        c2 = self.s.read_cart()
        self.assertEqual(c, c2)
        self.assertEqual(c.to_dict(), c2.to_dict())

        self.s.reset()
        c = self.s.read_cart()
        self.assertEqual(type(c), type(None))

In [10]:
suite = unittest.defaultTestLoader.loadTestsFromTestCase(Test_CartpoleServer)
unittest.TextTestRunner().run(suite)

.
----------------------------------------------------------------------
Ran 1 test in 0.006s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

## Unit Testing the GP Controller Default Behavior

We devise a default test for the `GP_TestDefaultController`. The default behavior comprises calling the OpenAPI generated test methods from the super class `TestDefaultController`.   

In [11]:
import unittest
class Test_GP_TestDefaultController(unittest.TestCase):

    @classmethod
    def setUpClass(cls):
        super().setUpClass()
        cls.gp_test=GP_TestDefaultController()
        
    def setUp(self):
        self.gp_test=Test_GP_TestDefaultController.gp_test

    def test_default(self):
        gp_cart = self.gp_test.test_cart_get()

In [12]:
suite = unittest.defaultTestLoader.loadTestsFromTestCase(Test_GP_TestDefaultController)
unittest.TextTestRunner().run(suite)

.

Content-Type: application/vnd.cartpole.cart+json
Content-Length: 64


{'direction': 'left', 'position': 1.1, 'velocity': 1.1}



----------------------------------------------------------------------
Ran 1 test in 0.142s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

## Unit Testing Monkey Patching

This section designs a few testcases for the `endpoint_config()` method in the `GP_TestDefaultController` class.

In [13]:
# functions replacing the default controller by Monkey Patching

from openapi_server.models import Cart

def testcase_all_correct():
    return Cart(position=1.1, velocity=2.2, direction='left')

def testcase_empty_return_object():
    return Cart()

def testcase_return_none():
    return None

def testcase_program_with_exception():
    raise Exception('Program raises an exception')

def testcase_return_string_type():
    return 'some string'

def testcase_return_obj_type():
    return type('obj', (object,), {'attribute' : 'attribute value'})

In [14]:
# the testcase with test methods

import unittest

class TestMonkeyPatching(unittest.TestCase):

    @classmethod
    def setUpClass(cls):
        super().setUpClass()
        cls.gp_test=GP_TestDefaultController()

    def setUp(self):
        self.url_path = '/api/v1/cart'
        self.gp_test=TestMonkeyPatching.gp_test
        
    def test_all_correct(self):
        ret = self.gp_test.endpoint_config(self.url_path, 'get', testcase_all_correct)
        self.assertTrue(ret)
        gp_cart = self.gp_test.test_cart_get()
    
    def test_empty_return_object(self):
        ret = self.gp_test.endpoint_config(self.url_path, 'get', testcase_empty_return_object)
        self.assertTrue(ret)
        gp_cart = self.gp_test.test_cart_get()
    
    @unittest.expectedFailure
    def test_return_none(self):
        ret = self.gp_test.endpoint_config(self.url_path, 'get', testcase_return_none)
        self.assertTrue(ret)
        gp_cart = self.gp_test.test_cart_get()

    @unittest.expectedFailure
    def test_program_with_exception(self):
        ret = self.gp_test.endpoint_config(self.url_path, 'get', testcase_program_with_exception)
        self.assertTrue(ret)
        gp_cart = self.gp_test.test_cart_get()
        
    @unittest.expectedFailure
    def test_return_string_type(self):
        ret = self.gp_test.endpoint_config(self.url_path, 'get', testcase_return_string_type)
        self.assertTrue(ret)
        gp_cart = self.gp_test.test_cart_get()

    @unittest.expectedFailure
    def test_return_obj_type(self):
        ret = self.gp_test.endpoint_config(self.url_path, 'get', testcase_return_obj_type)
        self.assertTrue(ret)
        gp_cart = self.gp_test.test_cart_get()


In [15]:
suite = unittest.defaultTestLoader.loadTestsFromTestCase(TestMonkeyPatching)
unittest.TextTestRunner().run(suite)

..xxxhttp://localhost/api/v1/cart validation error: 'some string' is not of type 'object'

Failed validating 'type' in schema:
    {'components': {'schemas': {'Cart': {'properties': {'direction': {'enum': ['left',
                                                                               'right'],
                                                                      'type': 'string'},
                                                        'position': {'format': 'double',
                                                                     'type': 'number'},
                                                        'velocity': {'format': 'double',
                                                                     'type': 'number'}},
                                         'type': 'object'},
                                'Direction': {'properties': {'direction': {'enum': ['left',
                                                                                    'right'],
       

Content-Type: application/vnd.cartpole.cart+json
Content-Length: 64


{'direction': 'left', 'position': 1.1, 'velocity': 2.2}
Content-Type: application/vnd.cartpole.cart+json
Content-Length: 3


{}



----------------------------------------------------------------------
Ran 6 tests in 0.200s

OK (expected failures=4)


<unittest.runner.TextTestResult run=6 errors=0 failures=0>